# Bitcoin & Twitter

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from datetime import datetime
from pyspark import SparkContext

#BITCOIN

# Unix Timestamp - Date represented as epoc value
# Date - date and time when the data point was collected
# Symbol - Symbol of the currency
# Open - Open value of the currency
# High - Highest value of currency in the given minute
# Low - Lowest value of currency in the given minute
# Close - Close value of the currency in the given minute
# Volume - Volume of the currency transacted in the given minute.

# TWITTER

# user_name	The name of the user, as they’ve defined it.
# user_location	The user-defined location for this account’s profile.
# user_description	The user-defined UTF-8 string describing their account.
# user_created	Time and date, when the account was created.
# user_followers	The number of followers an account currently has.
# user_friends	The number of friends a account currently has.
# user_favourites	The number of favorites a account currently has
# user_verified	When true, indicates that the user has a verified account
# date	UTC time and date when the Tweet was created
# text	The actual UTF-8 text of the Tweet
# hashtags	All the other hashtags posted in the tweet along with #Bitcoin & #btc
# source	Utility used to post the Tweet, Tweets from the Twitter website have a source value - web
# is_retweet	Indicates whether this Tweet has been Retweeted by the authenticating user.
# sc = SparkContext("local", "main")
bitcoin_data = (sc.textFile('data/bitcoinvalue.csv').filter(lambda l: 'close' not in l))
twitter_data = (sc.textFile('data/tweets.csv').filter(lambda l: 'user_name' not in l))
print(bitcoin_data.collect())
print(twitter_data.collect())